In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os

os.environ['R_HOME']="C:\\Users\\s9554\\Anaconda3\\envs\\stats\\Lib\\R"

import rpy2.robjects.packages as rpackages
import rpy2.robjects as ro

# pandas data to R data
from rpy2.robjects import pandas2ri

C:\Users\s9554\Anaconda3\envs\stats\lib\site-packages\rpy2-2.9.4-py3.7-win-amd64.egg\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [2]:
pandas2ri.activate()

In [3]:
reg = rpackages.importr("mcr").mcreg

def passing_bablok_reg(x,y):
    x = pandas2ri.py2ri(x)
    y = pandas2ri.py2ri(y)
    res = reg(x, y, method_reg="PaBa")

    print(res.slots['para'])

In [4]:
df = pd.read_csv("wbc.csv")
df=df.iloc[:,0:6].dropna()
df

C:\Users\s9554\Anaconda3\envs\stats\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,NO,TYPE,UF-5000,XN-9000,XN-9000(H)
0,1.0,A062187955,Pleural fluid,985.4,2207.0,2310.0
1,2.0,A062188014,CSF,1.0,1.0,1.0
3,4.0,A065190261,Pleural fluid,468.7,854.0,874.0
4,5.0,0410660187,Pleural fluid,432.5,1037.0,1128.0
5,6.0,A062186689,Bronchial,664.0,661.0,706.0
...,...,...,...,...,...,...
294,295.0,A062689766,Bronchial,841.8,836.0,872.0
295,296.0,A062699976,Pleural fluid,1672.0,1666.0,1668.0
296,297.0,A062699923,Pleural fluid,223.5,215.0,216.0
298,299.0,1840901800,Pleural fluid,516.6,429.0,574.0


In [5]:
df = df[~df['Unnamed: 0'].isin([1,4,5,8, 22, 70, 85, 94, 105, 127, 132, 218, 223, 237, 282, 290])]
df[df['TYPE']=="Synovial fluid"]

df.to_csv("wbc_mod.csv")

In [6]:
df['TYPE'].value_counts()


Pleural fluid     114
Bronchial          96
Ascites            37
CSF                17
Synovial fluid     10
Name: TYPE, dtype: int64

In [7]:
# df2 = df[df['TYPE']!="Synovial fluid"]
print("All data")
res = passing_bablok_reg(df['UF-5000'], df["XN-9000(H)"])


All data
                EST SE       LCI       UCI

Intercept -1.421136 NA -6.482936 0.4364015

Slope      1.079835 NA  1.049138 1.1256586



In [8]:
grouped = df.groupby("TYPE")

for k in grouped.groups.keys():
    print(k)
    sub_df = grouped.get_group(k)
    res = passing_bablok_reg(sub_df['UF-5000'], sub_df["XN-9000(H)"])
    print()
    

Ascites
                EST SE        LCI       UCI

Intercept -4.184500 NA -18.389539 0.8029884

Slope      1.084569 NA   1.025286 1.2754756


Bronchial
               EST SE         LCI       UCI

Intercept 3.936232 NA -44.6474798 51.723005

Slope     1.045706 NA   0.9631521  1.109575


CSF
                  EST SE        LCI       UCI

Intercept 0.007657424 NA -0.7910795 0.7597173

Slope     0.992342576 NA  0.9545334 1.2087912


Pleural fluid
                 EST SE        LCI      UCI

Intercept -12.001205 NA -27.999288 1.585035

Slope       1.127711 NA   1.057832 1.221716


Synovial fluid
                 EST SE          LCI       UCI

Intercept -624.09511 NA -8038.141169 -8.073881

Slope        1.93572 NA     1.159238  4.090844




In [9]:
dc = pd.read_csv("dc.csv")
dc = dc.dropna()
dc

,Unnamed: 0,TYPE,UF5000-MNC,UF5000-PMN,XN9000-MNC,XN9000-PMN
0,1,Pleural fluid,76.6,23.4,54.3,45.7
1,2,Synovial fluid,17.2,82.8,9.0,91.0
3,4,Pleural fluid,88.2,11.8,92.2,7.8
4,5,Bronchial,54.4,45.6,50.6,49.4
5,6,Ascites,98.0,2.0,77.5,22.5
...,...,...,...,...,...,...
263,264,Pleural fluid,97.1,2.9,93.9,6.1
264,265,Pleural fluid,46.4,53.6,33.7,66.3
265,266,Pleural fluid,55.5,44.5,48.9,51.1
266,267,Pleural fluid,79.6,20.4,64.0,36.0


In [10]:
print("all MNC")
passing_bablok_reg(dc['UF5000-MNC'], dc['XN9000-MNC'])

print("all PMN")
passing_bablok_reg(dc['UF5000-PMN'], dc['XN9000-PMN'])

all MNC
                EST SE         LCI       UCI

Intercept -9.415079 NA -12.5805577 -6.798068

Slope      1.031746 NA   0.9939521  1.074258

all PMN
               EST SE       LCI      UCI

Intercept 6.346825 NA 4.2234121 8.427959

Slope     1.031746 NA 0.9911121 1.073752



In [11]:
grouped_dc = dc.groupby("TYPE")

for k in grouped_dc.groups.keys():
    sub_df = grouped_dc.get_group(k)
    print(f"{k}-MNC")
    res = passing_bablok_reg(sub_df["UF5000-MNC"], sub_df['XN9000-MNC'])
    
    print(f"{k}-PMN")
    res = passing_bablok_reg(sub_df['UF5000-PMN'], sub_df['XN9000-PMN'])
    print()

Ascites-MNC
                 EST SE          LCI       UCI

Intercept -29.478104 NA -100.9114286 -9.027303

Slope       1.218962 NA    0.9693252  2.026561

Ascites-PMN
               EST SE        LCI       UCI

Intercept 7.578776 NA 0.04278146 15.784701

Slope     1.219237 NA 0.96845426  2.026455


Bronchial-MNC
                EST SE         LCI       UCI

Intercept -8.637500 NA -14.1120683 -4.300000

Slope      1.041667 NA   0.9245809  1.156464

Bronchial-PMN
               EST SE        LCI       UCI

Intercept 4.470833 NA -2.5110179 13.262023

Slope     1.041667 NA  0.9231619  1.160016


CSF-MNC
                EST SE         LCI       UCI

Intercept -3.843255 NA -56.6583026 15.596875

Slope      1.038433 NA   0.6092545  1.601476

CSF-PMN
               EST SE        LCI       UCI

Intercept 0.000000 NA -6.4959202 29.650900

Slope     1.038433 NA  0.5824935  1.712264


Pleural fluid-MNC
                 EST SE        LCI       UCI

Intercept -15.707229 NA -24.814765 -9.820797

Slo